In [2]:
from greedy_solver import GreedyFJSSPSolver, GreedyFJSSPWSolver


In [3]:
job_sequence = [0,0,0,1,1,2,2,2,3,3,3,3,4,4,4,5,5]
jobs = set(job_sequence)
machines = [0, 1, 2, 3]
durations = [
    [5,10,0,0],
    [0,5,10,0],
    [0,0,5,10],
    [0,2,10,5],
    [0,10,5,15],
    [10,5,3,0],
    [5,10,0,0],
    [0,5,10,3],
    [3,0,5,10],
    [0,1,10,5],
    [0,1,5,10],
    [0,5,10,2],
    [5,10,20,0],
    [20,5,10,0],
    [0,4,5,10],
    [0,4,10,5],
    [3,10,5,0]
]
fjssp_solver = GreedyFJSSPSolver(durations, job_sequence)
sequence, machines = fjssp_solver.solve()

In [4]:
sequence

[1, 3, 3, 3, 3, 2, 5, 5, 2, 2, 1, 0, 4, 0, 4, 4, 0]

In [5]:
job_sequence = [0,0,0,1,1,2,2,2,3,3,3,3,4,4,4,5,5]
jobs = set(job_sequence)
machines = [0, 1, 2, 3]
durations = [
    [[5,10,0,0] ,[5,10,2,0]   ],
    [[0,5,10,0] ,[0,15,10,0]   ],
    [[0,0,5,10] ,[0,0,25,10]   ],
    [[0,2,10,5] ,[0,22,10,5]   ],
    [[0,10,5,15],[3,10,5,15]   ],
    [[10,5,3,0] ,[10,5,32,0]   ],
    [[5,10,0,0] ,[5,10,0,0]   ],
    [[0,5,10,3] ,[0,5,10,31]   ],
    [[3,0,5,10] ,[35,0,5,10]   ],
    [[0,1,10,5] ,[0,11,10,5]   ],
    [[0,1,5,10] ,[0,17,5,10]   ],
    [[0,5,10,2] ,[0,5,10,0]   ],
    [[5,10,20,0],[0,10,20,0]  ],
    [[20,5,10,0],[20,51,10,0]   ],
    [[0,4,5,10] ,[0,14,5,10]   ],
    [[0,4,10,5] ,[0,14,10,5]   ],
    [[3,10,5,0] ,[31,10,5,0]   ]
]
fjssp_w_solver = GreedyFJSSPWSolver(durations, job_sequence)
sequence, machines, workers = fjssp_w_solver.solve()

In [6]:
sequence

[0, 1, 3, 3, 3, 3, 1, 2, 5, 5, 4, 4, 4, 2, 2, 0, 0]

In [7]:
from benchmark_parser import WorkerBenchmarkParser
parser = WorkerBenchmarkParser()
encoding = parser.parse_benchmark(r'C:\Users\huda\Documents\GitHub\scheduling_model_jrc\code\upgrades\benchmarks_with_workers\6_Fattahi_20_workers.fjs')

In [8]:
solver = GreedyFJSSPWSolver(encoding.durations(), encoding.job_sequence())
s, m, w = solver.solve()

In [9]:
import numpy as np

def makespan(start_times : list[int], machine_assignments : list[int], worker_assignments : list[int], durations : list[list[list[int]]]) -> float:
    # NOTE: assume first operation to start at t = 0
    return np.max([start_times[i] + durations[i][machine_assignments[i]][worker_assignments[i]] for i in range(len(start_times))])

def workload_balance(machine_assignments : list[int], worker_assignments : list[int], durations : list[list[list[int]]]) -> float:
    n_workers = max(worker_assignments)
    working_time = [0] * n_workers
    for i in range(len(worker_assignments)):
        working_time[worker_assignments[i]] += durations[i][machine_assignments[i]][worker_assignments[i]]
    mean_working_time = np.mean(working_time)
    result = 0.0
    for i in range(len(working_time)):
        result += np.pow((mean_working_time - working_time), 2)
    return result

def translate(sequence : list[int], machines : list[int], workers : list[int], durations : list[list[list[int]]]) -> tuple[list[int], list[int], list[int]]:
    
    class TimeSlot:
        def __init__(self) -> None:
            self.start = 0
            self.end = 0

        def __init__(self, start : int, end : int) -> None:
            self.start = start
            self.end = end

        def overlaps(self, other) -> bool:
            return self.contains(other.start) or self.contains(other.end) or other.contains(self.start) or other.contains(self.end)

        def contains(self, time : int) -> bool:
            return time >= self.start and time <= self.end
    
    def get_start_index(job : int, job_sequence : list[int]) -> int:
        for i in range(len(job_sequence)):
            if job_sequence[i] == job:
                return i
        return None

    def earliest_fit(times : list[TimeSlot], slot : TimeSlot) -> TimeSlot:
        for i in range(1, len(times)):
            if times[i-1].end >= slot.start and times[i].start >= slot.end:
                return times[i-1]
        return times[-1]

    job_sequence = sorted(sequence)
    jobs = set(job_sequence)
    start_indices = [get_start_index(job, job_sequence) for job in sorted(list(jobs))]
    n_operations = len(durations)
    n_machines = len(durations[0])
    n_workers = len(durations[0][0])
    next_operation : list[int] = [0] * len(jobs)
    end_on_machines : list[TimeSlot] = [[TimeSlot(0, 0)] for _ in range(n_machines)]
    end_of_workers : list[TimeSlot] = [[TimeSlot(0, 0)] for _ in range(n_workers)]
    start_times : list[int] = [0] * n_operations
    end_times : list[int] = [0] * n_operations
    for i in range(n_operations):
        job = sequence[i]
        operation = next_operation[job]
        next_operation[job] += 1
        start_index = start_indices[job] + operation
        machine = machines[i]
        worker = workers[i]
        duration = durations[i][machine][worker]
        if duration == 0:
            raise Exception("Invalid solution")
        offset = 0
        if operation > 0:
            if end_times[start_index - 1] > offset:
                offset = end_times[start_index - 1]
        if len(end_on_machines[machine]) > 0 and end_on_machines[machine][-1].end >= offset:
            offset = end_on_machines[machine][-1].end
        if len(end_of_workers[worker]) > 0:
            worker_earliest : TimeSlot = earliest_fit(end_of_workers[worker], TimeSlot(offset, offset + duration))
            if worker_earliest.end >= offset:
                offset = worker_earliest.end
        start_times[start_index] = offset
        end_times[start_index] = offset + duration
        end_on_machines[machine].append(TimeSlot(offset, offset + duration))
        end_of_workers[worker].append(TimeSlot(offset, offset + duration))
        end_on_machines[machine].sort(key=lambda x: x.start) # should be sorted anyway
        end_of_workers[worker].sort(key=lambda x: x.start)
    return start_times, machines, workers

In [10]:
start_times, machines, workers = translate(s, m, w, encoding.durations())

In [11]:
makespan(start_times, machines, workers, encoding.durations())

np.int64(4437)

In [ ]:
import os
base_path = r'C:\Users\huda\Documents\GitHub\scheduling_model_jrc\code\upgrades\benchmarks_with_workers\\'
out_path = r'C:\Users\huda\Desktop\experiments\greedy\greedy_results.txt'
instances = os.listdir(base_path)
results = []
for instance in instances:
    parser = WorkerBenchmarkParser()
    encoding = parser.parse_benchmark(f'{base_path}{instance}')
    print(f'{len(results)+1}- Solving: {instance}')
    solver = GreedyFJSSPWSolver(encoding.durations(), encoding.job_sequence())
    s, m, w = solver.solve()
    start_times, machines, workers = translate(s, m, w, encoding.durations())
    c = makespan(start_times, machines, workers, encoding.durations())
    results.append(f'{instance};{c};{s};{m};{w}\n')
with open(out_path, 'a') as f:
    f.writelines(results)

1- Solving: 0_BehnkeGeiger_10_workers.fjs
2- Solving: 0_BehnkeGeiger_11_workers.fjs
3- Solving: 0_BehnkeGeiger_12_workers.fjs
4- Solving: 0_BehnkeGeiger_13_workers.fjs
5- Solving: 0_BehnkeGeiger_14_workers.fjs
6- Solving: 0_BehnkeGeiger_15_workers.fjs
7- Solving: 0_BehnkeGeiger_16_workers.fjs
8- Solving: 0_BehnkeGeiger_17_workers.fjs
9- Solving: 0_BehnkeGeiger_18_workers.fjs
10- Solving: 0_BehnkeGeiger_19_workers.fjs
11- Solving: 0_BehnkeGeiger_1_workers.fjs
12- Solving: 0_BehnkeGeiger_20_workers.fjs
13- Solving: 0_BehnkeGeiger_21_workers.fjs
14- Solving: 0_BehnkeGeiger_22_workers.fjs
15- Solving: 0_BehnkeGeiger_23_workers.fjs
16- Solving: 0_BehnkeGeiger_24_workers.fjs
17- Solving: 0_BehnkeGeiger_25_workers.fjs
18- Solving: 0_BehnkeGeiger_26_workers.fjs
19- Solving: 0_BehnkeGeiger_27_workers.fjs
20- Solving: 0_BehnkeGeiger_28_workers.fjs
21- Solving: 0_BehnkeGeiger_29_workers.fjs
22- Solving: 0_BehnkeGeiger_2_workers.fjs
23- Solving: 0_BehnkeGeiger_30_workers.fjs
24- Solving: 0_BehnkeG